In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

2021-10-14 11:08:47.962541: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-14 11:08:47.962584: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 一二问

In [3]:
Spot_A_FirstPrediction = pd.read_excel("/home/competition/math_modeling/questions/B/1.xlsx",sheet_name="监测点A逐小时污染物浓度与气象一次预报数据")

In [5]:
Spot_A_FirstPrediction.head()

,模型运行日期,预测时间,地点,近地2米温度（℃）,地表温度（K）,比湿（kg/kg）,湿度（%）,近地10米风速（m/s）,近地10米风向（°）,雨量（mm）,...,潜热通量（W/m²）,长波辐射（W/m²）,短波辐射（W/m²）,地面太阳能辐射（W/m²）,SO2小时平均浓度(μg/m³),NO2小时平均浓度(μg/m³),PM10小时平均浓度(μg/m³),PM2.5小时平均浓度(μg/m³),O3小时平均浓度(μg/m³),CO小时平均浓度(mg/m³)
0,2020-07-23,2020-07-23 00:00:00,监测点A,29.8890,304.016,0.018870,66.7409,4.16382,162.577,0.000000,...,0.94818,428.278,0.0,0.0,2.401510,20.9208,8.17336,5.27729,8.78723,0.124491
1,2020-07-23,2020-07-23 01:00:00,监测点A,29.8736,303.739,0.017556,62.1551,4.65267,171.978,0.000000,...,1.14987,427.531,0.0,0.0,1.448340,14.8144,6.49054,4.33106,12.74530,0.109056
2,2020-07-23,2020-07-23 02:00:00,监测点A,29.6471,303.419,0.017874,64.1760,4.10031,172.013,0.000000,...,1.01616,427.428,0.0,0.0,1.271610,13.9154,6.86679,4.40045,12.22960,0.105957
3,2020-07-23,2020-07-23 03:00:00,监测点A,29.4555,303.419,0.018935,68.7958,2.44317,168.135,0.047224,...,1.89003,442.472,0.0,0.0,0.467429,11.1535,5.25900,3.35261,13.78000,0.101764
4,2020-07-23,2020-07-23 04:00:00,监测点A,28.5189,302.987,0.019881,76.5791,2.57759,207.884,8.260020,...,6.53753,458.394,0.0,0.0,0.574856,13.9989,6.05979,3.59303,9.96333,0.104536


In [6]:
Spot_A_FirstPrediction.describe()

,近地2米温度（℃）,地表温度（K）,比湿（kg/kg）,湿度（%）,近地10米风速（m/s）,近地10米风向（°）,雨量（mm）,云量,边界层高度（m）,大气压（Kpa）,...,潜热通量（W/m²）,长波辐射（W/m²）,短波辐射（W/m²）,地面太阳能辐射（W/m²）,SO2小时平均浓度(μg/m³),NO2小时平均浓度(μg/m³),PM10小时平均浓度(μg/m³),PM2.5小时平均浓度(μg/m³),O3小时平均浓度(μg/m³),CO小时平均浓度(mg/m³)
count,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,...,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,25416.000000,2.541600e+04,25416.000000
mean,24.941367,301.906307,0.013027,59.271910,3.927604,123.315270,0.391002,0.460008,674.963830,101.190651,...,16.962561,395.170426,215.264752,258.667851,8.095342,57.976588,33.951973,27.052738,3.289282e+01,0.298747
std,5.972074,8.613403,0.005130,16.499209,1.696326,79.249468,1.991600,0.326640,450.299871,0.651010,...,19.651134,49.883333,276.857248,332.679032,7.977947,34.019993,28.272897,24.408726,3.549418e+01,0.231951
min,3.413970,275.448000,0.000647,8.546440,0.012006,0.002380,0.000000,0.000000,19.774900,99.664800,...,-2.089550,230.671000,0.000000,0.000000,0.003020,7.685690,0.538817,0.410555,2.052830e-14,0.089578
25%,21.211475,296.083750,0.009315,47.224525,2.711615,56.523125,0.000000,0.150826,330.467500,100.631000,...,1.073412,360.149500,0.000000,0.000000,2.695410,31.543275,11.532400,7.621767,4.135275e+00,0.163849
50%,25.947200,301.986000,0.013289,59.458100,3.834030,130.049500,0.000000,0.439354,549.772000,101.190000,...,6.617565,407.791000,31.445800,37.786050,5.527160,50.482150,25.355650,17.991850,2.122600e+01,0.229687
75%,29.278250,307.547000,0.017701,72.739450,5.059253,172.048250,0.000000,0.774837,969.257750,101.689250,...,30.767725,435.506750,422.042750,507.138000,10.960800,78.987875,49.329875,41.128100,5.217420e+01,0.349102
max,37.471400,323.987000,0.023138,99.149200,12.712100,359.995000,86.203000,1.000000,2465.210000,102.925000,...,77.258600,491.217000,889.015000,1068.260000,134.948000,610.844000,253.909000,190.873000,4.151800e+02,3.214280
